In [8]:
import os
import numpy as np
import pandas as pd
import librosa
import librosa.display
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import torch
import torch.nn as nn
import torch.optim as optim
import sounddevice as sd
from scipy.io.wavfile import write
import tempfile
from ipywidgets import Dropdown, Button, Output, VBox
import IPython.display as ipd

# Function to list audio files in a directory
def list_audio_files(directory):
    return [file for file in os.listdir(directory) if file.endswith('.wav')]

# Function to extract MFCC features
def feature_extraction(file_path):
    x, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
    mfcc = np.mean(librosa.feature.mfcc(y=x, sr=sample_rate, n_mfcc=22).T, axis=0)
    mfcc_labels = [f'MFCC_Coefficient_{i+1}' for i in range(len(mfcc))]
    return mfcc, mfcc_labels

# Directories for Healthy Controls (HC) and PwPD
hc_directory = r'C:\Users\S M N RAZA\Downloads\23849127\HC_AH\HC_AH'
pwpd_directory = r'C:\Users\S M N RAZA\Downloads\23849127\PD_AH\PD_AH'

# Extract MFCC features and prepare the dataset
mfcc_df = pd.DataFrame()
for label, directory in [('HC', hc_directory), ('PwPD', pwpd_directory)]:
    for audio_file in list_audio_files(directory):
        audio_path = os.path.join(directory, audio_file)
        mfcc_features, mfcc_labels = feature_extraction(audio_path)
        temp_df = pd.DataFrame(mfcc_features.reshape(1, -1), columns=mfcc_labels)
        temp_df['Audio_File'] = audio_file
        temp_df['Label'] = label
        mfcc_df = pd.concat([mfcc_df, temp_df], ignore_index=True)

# Prepare data for model training
X = mfcc_df.drop(columns=['Audio_File', 'Label']).values
y = mfcc_df['Label'].map({'HC': 0, 'PwPD': 1}).values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape data for CNN
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define CNN model in PyTorch
class CNNModel(nn.Module):
    def __init__(self, input_shape):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=3)
        self.pool = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3)
        
        # Calculate flattened size dynamically
        conv1_output_size = input_shape[0] - (3 - 1)
        pool1_output_size = conv1_output_size // 2
        conv2_output_size = pool1_output_size - (3 - 1)
        pool2_output_size = conv2_output_size // 2
        
        self.flatten_size = 64 * pool2_output_size
        self.fc1 = nn.Linear(self.flatten_size, 128)
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

# Initialize CNN model
input_shape = (X_train_cnn.shape[1], 1)
cnn_model = CNNModel(input_shape)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_cnn, dtype=torch.float32).permute(0, 2, 1)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_cnn, dtype=torch.float32).permute(0, 2, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Train the CNN model
epochs = 10
batch_size = 32
for epoch in range(epochs):
    cnn_model.train()
    permutation = torch.randperm(X_train_tensor.size(0))
    epoch_loss = 0.0
    for i in range(0, X_train_tensor.size(0), batch_size):
        indices = permutation[i:i + batch_size]
        batch_X, batch_y = X_train_tensor[indices], y_train_tensor[indices]
        
        optimizer.zero_grad()
        outputs = cnn_model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.4f}")

# Evaluate the model
cnn_model.eval()
with torch.no_grad():
    predictions = cnn_model(X_test_tensor).squeeze().round()
    print("CNN Classification Report:")
    print(classification_report(y_test_tensor.numpy(), predictions.numpy()))
    print("CNN Confusion Matrix:")
    print(confusion_matrix(y_test_tensor.numpy(), predictions.numpy()))

# Save the trained model
torch.save(cnn_model.state_dict(), 'cnn_parkinsons_model.pth')
print("CNN model saved as cnn_parkinsons_model.pth.")

# Function to record live audio
def record_audio(duration=5, sample_rate=22050):
    print("Recording...")
    audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
    sd.wait()
    print("Recording complete.")
    temp_file = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    write(temp_file.name, sample_rate, audio)
    return temp_file.name

# Function to classify a single audio file
def classify_audio(audio_file, model, rfc_model):
    mfcc_features, _ = feature_extraction(audio_file)
    input_tensor = torch.tensor(mfcc_features.reshape(1, -1, 1), dtype=torch.float32).permute(0, 2, 1)
    model.eval()
    with torch.no_grad():
        cnn_prediction = model(input_tensor).item()
    rfc_prediction = rfc_model.predict(mfcc_features.reshape(1, -1))[0]
    print(f"CNN Prediction: {'PwPD' if cnn_prediction > 0.5 else 'HC'}")
    print(f"Random Forest Prediction: {'PwPD' if rfc_prediction == 1 else 'HC'}")

# Load and test live audio classification
recorded_audio = record_audio()
classify_audio(recorded_audio, cnn_model, joblib.load('rfc_trained_model.joblib'))


Epoch 1/10, Loss: 2.5924
Epoch 2/10, Loss: 1.7626
Epoch 3/10, Loss: 2.0672
Epoch 4/10, Loss: 1.4257
Epoch 5/10, Loss: 1.5061
Epoch 6/10, Loss: 1.4988
Epoch 7/10, Loss: 1.2514
Epoch 8/10, Loss: 1.4356
Epoch 9/10, Loss: 1.2239
Epoch 10/10, Loss: 1.2733
CNN Classification Report:
              precision    recall  f1-score   support

         0.0       0.62      0.91      0.74        11
         1.0       0.00      0.00      0.00         6

    accuracy                           0.59        17
   macro avg       0.31      0.45      0.37        17
weighted avg       0.40      0.59      0.48        17

CNN Confusion Matrix:
[[10  1]
 [ 6  0]]
CNN model saved as cnn_parkinsons_model.pth.
Recording...
Recording complete.
CNN Prediction: PwPD
Random Forest Prediction: HC


c:\Users\S M N RAZA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
